In [ ]:
%%capture

# Upgrade pip
!pip install --upgrade pip
# Connectivity
!pip install psycopg2-binary  # PostgreSQL adapter
# !pip install snowflake-connector-python  # Snowflake connector
!pip install snowflake-connector-python==3.15.0 # Snowflake connector Older Version
!pip install snowflake-sqlalchemy  # Snowflake SQLAlchemy connector
!pip install warnings # Warnings management
# !pip install pyarrow # Serialization
!pip install keyring==23.11.0 # Key management
!pip install sqlalchemy==1.4.46 # SQLAlchemy
!pip install requests # HTTP requests
!pip install boto3 # AWS SDK
# !pip install slackclient # Slack API
!pip install oauth2client # Google Sheets API
!pip install gspread==5.9.0 # Google Sheets API
!pip install gspread_dataframe # Google Sheets API
!pip install google.cloud # Google Cloud
# Data manipulation and analysis
!pip install polars
!pip install pandas==2.2.1
!pip install numpy
# !pip install fastparquet
!pip install openpyxl # Excel file handling
!pip install xlsxwriter # Excel file handling
# Linear programming
!pip install pulp
# Date and time handling
!pip install --upgrade datetime
!pip install python-time
!pip install --upgrade pytz
# Progress bar
!pip install tqdm
# Database data types
!pip install db-dtypes
# Geospatial data handling
# !pip install geopandas
# !pip install shapely
# !pip install fiona
# !pip install haversine
# Plotting

# Modeling
!pip install statsmodels
!pip install scikit-learn

!pip install import-ipynb

In [58]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import calendar
import json
from datetime import date, timedelta
from oauth2client.service_account import ServiceAccountCredentials
import setup_environment_2
import importlib
import import_ipynb
import warnings
import boto3
import requests
warnings.filterwarnings("ignore")
importlib.reload(setup_environment_2)
setup_environment_2.initialize_env()
import os

/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
today = datetime.today()- timedelta(days=4)
month_start = today.replace(day=1)
first_part = (today - month_start).days

last_day = calendar.monthrange(today.year, today.month)[1]
second_part = (last_day - today.day)+1

In [4]:
def query_snowflake(query, columns=[]):
    import os
    import snowflake.connector
    import numpy as np
    import pandas as pd
    con = snowflake.connector.connect(
        user =  os.environ["SNOWFLAKE_USERNAME"],
        account= os.environ["SNOWFLAKE_ACCOUNT"],
        password= os.environ["SNOWFLAKE_PASSWORD"],
        database =os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        if len(columns) == 0:
            out = pd.DataFrame(np.array(cur.fetchall()))
        else:
            out = pd.DataFrame(np.array(cur.fetchall()),columns=columns)
        return out
    except Exception as e:
        print("Error: ", e)
    finally:
        cur.close()
        con.close()

In [5]:
query = '''
SHOW PARAMETERS LIKE 'TIMEZONE'
'''
x  = query_snowflake(query)
zone_to_use = x[1].values[0]
zone_to_use

'America/Los_Angeles'

## Prodcut Selection

In [6]:
command_string = f'''
with last_update as (
select  DATE_PART('hour', max_date) * 60 + DATE_PART('minute', max_date) AS total_minutes
from (
select max(created_at) as max_date from sales_orders
)

),
 predicted_rr  as (
select product_id,warehouse_id,rr,date
from Finance.PREDICTED_RUNNING_RATES
where date >= CURRENT_DATE
qualify date = max(date)over(partition by product_id,warehouse_id)
),
days_stocks as (
select timestamp::date as date ,product_id,warehouse_id,avg(in_stock) as in_stock_perc,avg(case when date_part('hour',timestamp) =date_part('hour',current_timestamp)-1 then  in_stock end) as last_hour_stocks
from (
select timestamp,product_id,warehouse_id,case when AVAILABLE_STOCK > 0 then 1 else 0 end as in_stock
from materialized_views.STOCK_SNAP_SHOTS_RECENT sss
where sss.timestamp::date >= date_trunc('month',current_date - 60)
and date_part('hour',sss.timestamp)<date_part('hour',CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) 
and warehouse_id in (1,8,170,236,337,339,401,501,632,703,797,962)
)
group by all 
),
base as (
select *, row_number()over(partition by retailer_id order by priority) as rnk 
from (
select x.*,TAGGABLE_ID as retailer_id 
from (
select id as cohort_id,name as cohort_name,priority,dynamic_tag_id 
from cohorts 
where is_active = 'true'
and id in (700,701,702,703,704,1123,1124,1125,1126)
) x 
join DYNAMIC_TAGgables dt on x.dynamic_tag_id = dt.dynamic_tag_id
)
qualify rnk = 1 
),
sales_data as (
SELECT  DISTINCT
		so.created_at::date as date,
		pso.warehouse_id as warehouse_id,
		pso.product_id,
		CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,
		brands.name_ar as brand, 
		categories.name_ar as cat,
        sum(pso.total_price) as all_day_nmv,
		sum(case when (date_part('hour',so.created_at)*60 + DATE_PART('minute', so.created_at))< (select * from last_update) then pso.total_price end) as uth_nmv,
		sum(case when (date_part('hour',so.created_at)*60 + DATE_PART('minute', so.created_at))
		between (select * from last_update) -60 
		and (select * from last_update)
		then pso.total_price end) as last_hour_nmv,


FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN products on products.id=pso.product_id
JOIN brands on products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN finance.all_cogs f  ON f.product_id = pso.product_id
                        AND f.from_date::date <= so.created_at ::date
                        AND f.to_date::date > so.created_at ::date
JOIN product_units ON product_units.id = products.unit_id 
JOIN materialized_views.retailer_polygon on materialized_views.retailer_polygon.retailer_id=so.retailer_id
JOIN districts on districts.id=materialized_views.retailer_polygon.district_id
JOIN cities on cities.id=districts.city_id
join states on states.id=cities.state_id
join regions on regions.id=states.region_id  

WHERE   True
    AND so.created_at ::date >= date_trunc('month',current_date - 60)
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0

GROUP BY ALL
order by date desc
),
data as (
select * , 1/nullif((0.3*week_distance+0.1*month_distance+0.6*day_distance),0) as distance
from (
select * ,
floor((DATE_PART('day', date) - 1) / 7 + 1) AS week_of_month,
DATE_PART('month', date) as month,
DATE_PART('DOW', date) AS day_number,

abs(floor((DATE_PART('day', current_date) - 1) / 7 + 1) - week_of_month)  as week_distance ,
abs(DATE_PART('month', current_date)- month) as month_distance,
abs(DATE_PART('DOW', current_date)- day_number) as day_distance
from (
select *, max(case when date = CURRENT_DATE then last_hour_stocks end) over(partition by product_id,warehouse_id) as current_stocks 
from (
select ds.*, all_day_nmv,
uth_nmv,
last_hour_nmv
from days_stocks ds
left join sales_data sd  on ds.product_id = sd.product_id and ds.warehouse_id = sd.warehouse_id and ds.date= sd.date
)
)
where current_stocks <> 0 
and (in_stock_perc = 1 or date = CURRENT_DATE)
)
),
current_state as (
select product_id,warehouse_id,AVAILABLE_STOCK,activation
from PRODUCT_WAREHOUSE
where IS_BASIC_UNIT = 1
)
select x.*,
cs.AVAILABLE_STOCK,
cs.activation,
coalesce(prr.rr,0) as rr,
case when coalesce(prr.rr,0) <>0 then cs.AVAILABLE_STOCK/coalesce(prr.rr,0) else cs.AVAILABLE_STOCK end  as doh ,
cs.AVAILABLE_STOCK*f.wac1  as stock_value
 from (
select product_id,warehouse_id,
coalesce(max(case when state = 'prev' then all_day_nmv end),0) as prev_all_day,
coalesce(max(case when state = 'prev' then uth_nmv end),0)  as prev_uth,
coalesce(max(case when state = 'prev' then last_hour_nmv end),0)  as prev_last_hour,

coalesce(max(case when state = 'current' then all_day_nmv end),0)  as current_all_day,
coalesce(max(case when state = 'current' then uth_nmv end),0)  as current_uth,
coalesce(max(case when state = 'current' then last_hour_nmv end),0)  as current_last_hour

from (
select 'current' as state,product_id,warehouse_id,all_day_nmv,uth_nmv,last_hour_nmv
from data
where date = CURRENT_DATE
union all 
(
select state,product_id,warehouse_id,
sum(all_day_nmv*distance)/sum(distance) as all_day_nmv,
sum(uth_nmv*distance)/sum(distance) as uth_nmv,
sum(last_hour_nmv*distance)/sum(distance) as last_hour_nmv
from(
select 'prev' as state,product_id,warehouse_id,all_day_nmv,uth_nmv,last_hour_nmv,distance
from data 
where date <> CURRENT_DATE
)
group by all 
)
)
group by all 
)x 
join current_state cs on x.product_id = cs.product_id and x.warehouse_id = cs.warehouse_id
left join predicted_rr prr on x.product_id = prr.product_id and x.warehouse_id = prr.warehouse_id 
join products p on p.id = x.product_id
join finance.all_cogs f on f.product_id = x.product_id and CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp()) between f.from_date and f.to_date 
where doh > 1 
and p.activation ='true'
and cs.activation = 'true'
and cs.AVAILABLE_STOCK * f.wac1 >= 1000
and prev_uth > 0
'''
product_data = query_snowflake(command_string, columns = ['product_id','warehouse_id','prev_all_day','prev_uth','prev_last_hour','current_all_day','current_uth','current_last_hour','available_stock','activation','rr','doh','stock_value'])
product_data.product_id = pd.to_numeric(product_data.product_id)
product_data.warehouse_id = pd.to_numeric(product_data.warehouse_id)
product_data.prev_all_day = pd.to_numeric(product_data.prev_all_day)
product_data.prev_uth = pd.to_numeric(product_data.prev_uth)
product_data.prev_last_hour = pd.to_numeric(product_data.prev_last_hour)
product_data.current_all_day = pd.to_numeric(product_data.current_all_day)
product_data.current_uth = pd.to_numeric(product_data.current_uth)
product_data.current_last_hour = pd.to_numeric(product_data.current_last_hour)
product_data.available_stock = pd.to_numeric(product_data.available_stock)
product_data.rr = pd.to_numeric(product_data.rr)
product_data.doh = pd.to_numeric(product_data.doh)
product_data.stock_value = pd.to_numeric(product_data.stock_value)

In [7]:
query = f'''
with last_update as (
select  DATE_PART('hour', max_date) * 60 + DATE_PART('minute', max_date) AS total_minutes
from (
select max(created_at) as max_date from sales_orders
)
),
base as (
select *, row_number()over(partition by retailer_id order by priority) as rnk 
from (
select x.*,TAGGABLE_ID as retailer_id 
from (
select id as cohort_id,name as cohort_name,priority,dynamic_tag_id 
from cohorts 
where is_active = 'true'
and id in (700,701,702,703,704,1123,1124,1125,1126)
) x 
join DYNAMIC_TAGgables dt on x.dynamic_tag_id = dt.dynamic_tag_id
)
qualify rnk = 1 
),
sales as (
SELECT 
		so.created_at::date as date,
		pso.warehouse_id as warehouse_id,
        sum(pso.total_price) as all_day_nmv,
		sum(case when (date_part('hour',so.created_at)*60 + DATE_PART('minute', so.created_at))< (select * from last_update) then pso.total_price end) as uth_nmv,
		sum(case when (date_part('hour',so.created_at)*60 + DATE_PART('minute', so.created_at))
		between (select * from last_update) -60 
		and (select * from last_update)
		then pso.total_price end) as last_hour_nmv,


FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN products on products.id=pso.product_id
JOIN brands on products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN finance.all_cogs f  ON f.product_id = pso.product_id
                        AND f.from_date::date <= so.created_at ::date
                        AND f.to_date::date > so.created_at ::date
JOIN product_units ON product_units.id = products.unit_id 
JOIN materialized_views.retailer_polygon on materialized_views.retailer_polygon.retailer_id=so.retailer_id
JOIN districts on districts.id=materialized_views.retailer_polygon.district_id
JOIN cities on cities.id=districts.city_id
join states on states.id=cities.state_id
join regions on regions.id=states.region_id  

WHERE   True
    AND so.created_at ::date between date_trunc('month',current_date - 60) and current_date -1 
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0

GROUP BY ALL
order by date desc
)
select warehouse_id,sum(uth_cntrb*distance)/sum(distance) as uth_cntrb
from (
select *, 1/nullif((0.3*week_distance+0.1*month_distance+0.6*day_distance),0) as distance
from(
select * ,uth_nmv/all_day_nmv as uth_cntrb,
floor((DATE_PART('day', date) - 1) / 7 + 1) AS week_of_month,
DATE_PART('month', date) as month,
DATE_PART('DOW', date) AS day_number,
abs(floor((DATE_PART('day', current_date) - 1) / 7 + 1) - week_of_month)  as week_distance ,
abs(DATE_PART('month', current_date)- month) as month_distance,
abs(DATE_PART('DOW', current_date)- day_number) as day_distance

from sales 
)
)
group by all 
'''
uth_cntrb = query_snowflake(query, columns = ['warehouse_id','uth_cntrb'])
uth_cntrb.warehouse_id = pd.to_numeric(uth_cntrb.warehouse_id)

In [8]:
query = f'''
WITH whs as (SELECT *
             FROM   (values
                            ('Cairo', 'El-Marg', 38,700),
                            ('Cairo', 'Mostorod', 1,700),
                            ('Giza', 'Barageel', 236,701),
                            ('Delta West', 'El-Mahala', 337,703),
                            ('Delta West', 'Tanta', 8,703),
                            ('Delta East', 'Mansoura FC', 339,704),
                            ('Delta East', 'Sharqya', 170,704),
                            ('Upper Egypt', 'Assiut FC', 501,1124),
                            ('Upper Egypt', 'Bani sweif', 401,1126),
                            ('Upper Egypt', 'Menya Samalot', 703,1123),
                            ('Upper Egypt', 'Sohag', 632,1125),
                            ('Alexandria', 'Khorshed Alex', 797,702),
							('Giza', 'Sakkarah', 962,701)
							
							)
                    x(region, wh, warehouse_id,cohort_id)),


local_prices as (
SELECT  case when cpu.cohort_id in (700,695) then 'Cairo'
             when cpu.cohort_id in (701) then 'Giza'
             when cpu.cohort_id in (704,698) then 'Delta East'
             when cpu.cohort_id in (703,697) then 'Delta West'
             when cpu.cohort_id in (696,1123,1124,1125,1126) then 'Upper Egypt'
             when cpu.cohort_id in (702,699) then 'Alexandria'
        end as region,
		cohort_id,
        pu.product_id,
		pu.packing_unit_id as packing_unit_id,
		pu.basic_unit_count,
        avg(cpu.price) as price
FROM    cohort_product_packing_units cpu
join    PACKING_UNIT_PRODUCTS pu on pu.id = cpu.product_packing_unit_id
WHERE   cpu.cohort_id in (700,701,702,703,704,696,695,698,697,699,1123,1124,1125,1126)
    and cpu.created_at::date<>'2023-07-31'
    and cpu.is_customized = true
	group by all 
),
live_prices as (
select region,cohort_id,product_id,pu_id as packing_unit_id,buc as basic_unit_count,NEW_PRICE as price
from materialized_views.DBDP_PRICES
where created_at = current_date
and DATE_PART('hour',CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) BETWEEN SPLIT_PART(time_slot, '-', 1)::int AND SPLIT_PART(time_slot, '-', 2)::int
and cohort_id in (700,701,702,703,704,696,695,698,697,699,1123,1124,1125,1126)
),
prices as (
select *
from (
    SELECT *, 1 AS priority FROM live_prices
    UNION ALL
    SELECT *, 2 AS priority FROM local_prices
)
QUALIFY ROW_NUMBER() OVER (PARTITION BY region,cohort_id,product_id,packing_unit_id ORDER BY priority) = 1
)
select warehouse_id,product_id,price 
from prices 
join whs on prices.cohort_id = whs.cohort_id
and basic_unit_count = 1 
'''
product_warehouse_price = query_snowflake(query, columns = ['warehouse_id','product_id','price'])
product_warehouse_price.warehouse_id = pd.to_numeric(product_warehouse_price.warehouse_id)
product_warehouse_price.product_id = pd.to_numeric(product_warehouse_price.product_id)
product_warehouse_price.price = pd.to_numeric(product_warehouse_price.price)

In [9]:
query = f'''
WITH whs as (SELECT *
             FROM   (values
                            ('Cairo', 'El-Marg', 38,700),
                            ('Cairo', 'Mostorod', 1,700),
                            ('Giza', 'Barageel', 236,701),
                            ('Delta West', 'El-Mahala', 337,703),
                            ('Delta West', 'Tanta', 8,703),
                            ('Delta East', 'Mansoura FC', 339,704),
                            ('Delta East', 'Sharqya', 170,704),
                            ('Upper Egypt', 'Assiut FC', 501,1124),
                            ('Upper Egypt', 'Bani sweif', 401,1126),
                            ('Upper Egypt', 'Menya Samalot', 703,1123),
                            ('Upper Egypt', 'Sohag', 632,1125),
                            ('Alexandria', 'Khorshed Alex', 797,702),
							('Giza', 'Sakkarah', 962,701)
							
							)
                    x(region, wh, warehouse_id,cohort_id)),
full_data as (
select products.id as product_id, region,warehouse_id
from products , whs 
where activation = 'true'
),				

MP as (
select region,product_id,
min(min_price) as min_price,
min(max_price) as max_price,
min(mod_price) as mod_price,
min(true_min) as true_min,
min(true_max) as true_max

from (
select mp.region,mp.product_id,mp.pu_id,
min_price/BASIC_UNIT_COUNT as min_price,
max_price/BASIC_UNIT_COUNT as max_price,
mod_price/BASIC_UNIT_COUNT as mod_price,
TRUE_MIN_PRICE/BASIC_UNIT_COUNT as true_min,
TRUE_MAX_PRICE/BASIC_UNIT_COUNT as true_max
from materialized_views.marketplace_prices mp 
join packing_unit_products pup on pup.product_id = mp.product_id and pup.packing_unit_id = mp.pu_id
)
group by all 
),
region_mapping AS (
    SELECT * 
	FROM 
	(	VALUES
        ('Delta East', 'Delta West'),
        ('Delta West', 'Delta East'),
        ('Alexandria', 'Cairo'),
        ('Alexandria', 'Giza'),
        ('Upper Egypt', 'Cairo'),
        ('Upper Egypt', 'Giza'),
		('Cairo','Giza'),
		('Giza','Cairo'),
		('Delta West', 'Cairo'),
		('Delta East', 'Cairo'),
		('Delta West', 'Giza'),
		('Delta East', 'Giza')
		)
    AS region_mapping(region, fallback_region)
)


select region,warehouse_id,product_id,
min(final_min_price) as final_min_price,
min(final_max_price) as final_max_price,
min(final_mod_price) as final_mod_price,
min(final_true_min) as final_true_min,
min(final_true_max) as final_true_max

from (
SELECT
distinct 
	w.region,
    w.warehouse_id,
	w.product_id,
    COALESCE(m1.min_price, m2.min_price) AS final_min_price,
    COALESCE(m1.max_price, m2.max_price) AS final_max_price,
    COALESCE(m1.mod_price, m2.mod_price) AS final_mod_price,
	COALESCE(m1.true_min, m2.true_min) AS final_true_min,
	COALESCE(m1.true_max, m2.true_max) AS final_true_max,
FROM full_data w
LEFT JOIN MP m1
    ON w.region = m1.region and w.product_id = m1.product_id
JOIN region_mapping rm
    ON w.region = rm.region
LEFT JOIN MP m2
    ON rm.fallback_region = m2.region
   AND w.product_id = m2.product_id
)
where final_min_price is not null 
group by all 
'''
marketplace = query_snowflake(query, columns = ['REGION','WAREHOUSE_ID','PRODUCT_ID','FINAL_MIN_PRICE','FINAL_MAX_PRICE','FINAL_MOD_PRICE','FINAL_TRUE_MIN','FINAL_TRUE_MAX'])
marketplace.columns = marketplace.columns.str.lower()
for col in marketplace.columns:
    marketplace[col] = pd.to_numeric(marketplace[col], errors='ignore')

In [10]:
query = f'''
select product_id,min(ben_soliman_basic_price) as ben_soliman_price
from (
select MAXAB_PRODUCT_ID as product_id,
MAXAB_BASIC_UNIT_COUNT as buc,
FINAL_PRICE_AFTER_CONVERSION as ben_soliman_price,
ben_soliman_price/buc as ben_soliman_basic_price,
max(INJECTION_DATE::date) OVER(PARTITION by product_id) as max_date
from materialized_views.savvy_mapping
where FINAL_PRICE_AFTER_CONVERSION is not null 
QUALIFY INJECTION_DATE::date = max_date
)
group by all 
'''

bensoliman = query_snowflake(query, columns = ['product_id','ben_soliman_basic_price'])
bensoliman.columns = bensoliman.columns.str.lower()
for col in bensoliman.columns:
    bensoliman[col] = pd.to_numeric(bensoliman[col], errors='ignore')     

In [11]:
query = f'''
WITH whs as (SELECT *
             FROM   (values
                            ('Cairo', 'El-Marg', 38,700),
                            ('Cairo', 'Mostorod', 1,700),
                            ('Giza', 'Barageel', 236,701),
                            ('Delta West', 'El-Mahala', 337,703),
                            ('Delta West', 'Tanta', 8,703),
                            ('Delta East', 'Mansoura FC', 339,704),
                            ('Delta East', 'Sharqya', 170,704),
                            ('Upper Egypt', 'Assiut FC', 501,1124),
                            ('Upper Egypt', 'Bani sweif', 401,1126),
                            ('Upper Egypt', 'Menya Samalot', 703,1123),
                            ('Upper Egypt', 'Sohag', 632,1125),
                            ('Alexandria', 'Khorshed Alex', 797,702),
							('Giza', 'Sakkarah', 962,701)
							
							)
                    x(region, wh, warehouse_id,cohort_id))
select product_id,x.region,warehouse_id,min(MARKET_PRICE) as min_scrapped,max(MARKET_PRICE) as max_scrapped,median(MARKET_PRICE) as median_scrapped
from (
select *,max(date)over(partition by region,product_id,competitor) as max_date
from MATERIALIZED_VIEWS.CLEANED_MARKET_PRICES
where date>= current_date -5
qualify date = max_date 
) x 
left join whs on whs.region = x.region
group by all 
'''

scrapped_prices = query_snowflake(query, columns = ['product_id','region','warehouse_id','min_scrapped','max_scrapped','median_scrapped'])
scrapped_prices.columns = scrapped_prices.columns.str.lower()
for col in scrapped_prices.columns:
    scrapped_prices[col] = pd.to_numeric(scrapped_prices[col], errors='ignore')          

In [12]:
query = f'''
select region,product_id,optimal_bm,MIN_BOUNDARY,MAX_BOUNDARY,MEDIAN_BM
from (
select region,product_id,target_bm,optimal_bm,MIN_BOUNDARY,MAX_BOUNDARY,MEDIAN_BM,max(created_at) over(partition by product_id,region) as max_date,created_at
from materialized_views.PRODUCT_STATISTICS
where created_at::date >= date_trunc('month',current_date - 60)
qualify max_date = created_at
)

'''
 
stats = query_snowflake(query, columns = ['region','product_id','optimal_bm','MIN_BOUNDARY','MAX_BOUNDARY','MEDIAN_BM'])
stats.columns = stats.columns.str.lower()
for col in stats.columns:
    stats[col] = pd.to_numeric(stats[col], errors='ignore')

In [13]:
query = f'''
select warehouse_id,region
from (
select * ,row_number()over(partition by warehouse_id order by nmv desc) as rnk 
from (
SELECT case when regions.id = 2 then cities.name_en else regions.name_en end as region,
	   pso.warehouse_id,
        sum(pso.total_price) as nmv



FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN materialized_views.retailer_polygon on materialized_views.retailer_polygon.retailer_id=so.retailer_id
JOIN districts on districts.id=materialized_views.retailer_polygon.district_id
JOIN cities on cities.id=districts.city_id
join states on states.id=cities.state_id
join regions on regions.id=states.region_id             

WHERE   True
    AND so.created_at ::date between current_date-31 and CURRENT_DATE-1
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0

GROUP BY ALL
)
qualify rnk = 1 
)
'''
warehouse_region = query_snowflake(query, columns = ['warehouse_id','region'])
warehouse_region.columns = warehouse_region.columns.str.lower()
for col in warehouse_region.columns:
    warehouse_region[col] = pd.to_numeric(warehouse_region[col], errors='ignore')    

In [14]:
query = f'''
SELECT DISTINCT cat, brand, margin as target_bm
FROM    performance.commercial_targets cplan
QUALIFY CASE WHEN DATE_TRUNC('month', MAX(DATE)OVER()) = DATE_TRUNC('month', CURRENT_DATE) THEN DATE_TRUNC('month', CURRENT_DATE)
ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') END = DATE_TRUNC('month', date)
'''
brand_cat_target  = query_snowflake(query, columns = ['cat','brand','target_bm'])
brand_cat_target.target_bm=pd.to_numeric(brand_cat_target.target_bm)

query = f'''
select cat,sum(target_bm *(target_nmv/cat_total)) as cat_target_margin
from (
select *,sum(target_nmv)over(partition by cat) as cat_total
from (
select cat,brand,avg(target_bm) as target_bm , sum(target_nmv) as target_nmv
from (
SELECT DISTINCT date,city as region,cat, brand, margin as target_bm,nmv as target_nmv
FROM    performance.commercial_targets cplan
QUALIFY CASE WHEN DATE_TRUNC('month', MAX(DATE)OVER()) = DATE_TRUNC('month', CURRENT_DATE) THEN DATE_TRUNC('month', CURRENT_DATE)
ELSE DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 month') END = DATE_TRUNC('month', date)
)
group by all
)
)
group by all 
'''
cat_target  = query_snowflake(query, columns = ['cat','cat_target_margin'])
cat_target.cat_target_margin=pd.to_numeric(cat_target.cat_target_margin)

query = f'''
SELECT  DIStinct  
		products.id as product_id,
		CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,
		brands.name_ar as brand, 
		categories.name_ar as cat,
		f.wac_p
from products 
JOIN brands on products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN finance.all_cogs f  ON f.product_id = products.id and CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp()) between f.from_date and f.to_date 
JOIN product_units ON product_units.id = products.unit_id 
'''
sku_info  = query_snowflake(query, columns = ['product_id','sku','brand','cat','wac_p'])
sku_info.product_id=pd.to_numeric(sku_info.product_id)
sku_info.wac_p=pd.to_numeric(sku_info.wac_p)

In [15]:
query = f'''
with last_update as (
select  DATE_PART('hour', max_date) * 60 + DATE_PART('minute', max_date) AS total_minutes
from (
select max(created_at) as max_date from sales_orders
)
)
SELECT  DISTINCT
		pso.warehouse_id,
		pso.product_id,
		coalesce(sum(case when DATE_PART('hour', so.created_at) * 60 + DATE_PART('minute', so.created_at)  between  (DATE_PART('hour', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) * 60 + DATE_PART('minute', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())))-120 and (DATE_PART('hour', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) * 60 + DATE_PART('minute', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())))-60 then pso.total_price end),0) as t_2_nmv,
		coalesce(sum(case when DATE_PART('hour', so.created_at) * 60 + DATE_PART('minute', so.created_at)  >  (DATE_PART('hour', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) * 60 + DATE_PART('minute', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())))-60 then pso.total_price end),0) as t_1_nmv,
		coalesce(avg(case when (DATE_PART('hour', so.created_at) * 60 + DATE_PART('minute', so.created_at)  between  (DATE_PART('hour', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) * 60 + DATE_PART('minute', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())))-120 and (DATE_PART('hour', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) * 60 + DATE_PART('minute', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())))-60) and (item_discount_value <> 0) then (pso.item_price/basic_unit_count) - (pso.item_discount_value/basic_unit_count) end),0) as t_2_price,
		coalesce(avg(case when (DATE_PART('hour', so.created_at) * 60 + DATE_PART('minute', so.created_at)  >  (DATE_PART('hour', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) * 60 + DATE_PART('minute', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())))-60) and (item_discount_value <> 0) then  (pso.item_price/basic_unit_count) - (pso.item_discount_value/basic_unit_count) end),0) as t_1_price



FROM product_sales_order pso 
JOIN sales_orders so ON so.id = pso.sales_order_id          

WHERE so.created_at::date = current_date 
and DATE_PART('hour', so.created_at) * 60 + DATE_PART('minute', so.created_at)  >=  (DATE_PART('hour', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())) * 60 + DATE_PART('minute', CONVERT_TIMEZONE('{zone_to_use}', 'Africa/Cairo', CURRENT_TIMEstamp())))-120
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0
GROUP BY ALL
'''
last_two_hours =  query_snowflake(query, columns = ['warehouse_id','product_id','t_2_nmv','t_1_nmv','t_2_price','t_1_price'])
last_two_hours.columns = last_two_hours.columns.str.lower()
for col in last_two_hours.columns:
    last_two_hours[col] = pd.to_numeric(last_two_hours[col], errors='ignore')     

In [16]:
query = '''

with main_data  as (
SELECT  DISTINCT
		pso.warehouse_id,
		pso.product_id,
		CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,
		brands.name_ar as brand, 
		categories.name_ar as cat,
		
        sum(pso.total_price) as nmv,
       sum(COALESCE(f.wac_p,0) * pso.purchased_item_count * pso.basic_unit_count) as cogs_p,
	   ((nmv-cogs_p)/nmv) as bm_p,


FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
--join COHORT_PRICING_CHANGES cpc on cpc.id = pso.COHORT_PRICING_CHANGE_id
JOIN products on products.id=pso.product_id
JOIN brands on products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN finance.all_cogs f  ON f.product_id = pso.product_id
                        AND f.from_date::date <= so.created_at ::date
                        AND f.to_date::date > so.created_at ::date
JOIN product_units ON product_units.id = products.unit_id 
JOIN materialized_views.retailer_polygon on materialized_views.retailer_polygon.retailer_id=so.retailer_id
JOIN districts on districts.id=materialized_views.retailer_polygon.district_id
JOIN cities on cities.id=districts.city_id
join states on states.id=cities.state_id
join regions on regions.id=states.region_id             

WHERE   True
    AND so.created_at ::date between  current_date - 5 and current_date -1 
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0

GROUP BY ALL
),
cp as (
select cat,brand,sum(nmv) as target_nmv ,avg(margin) as target_margin
from performance.commercial_targets 
where date  between '2025-10-01' and current_date - 1
group by all 
),
stocks as (					
select warehouse_id,warehouse,product_id,sum(stocks) as stocks
from (
		SELECT DISTINCT product_warehouse.warehouse_id,w.name as warehouse,
                product_warehouse.product_id,
                (product_warehouse.available_stock)::integer as stocks

        from  product_warehouse 
        JOIN products on product_warehouse.product_id = products.id
        JOIN product_units ON products.unit_id = product_units.id
		join warehouses w on w.id = product_warehouse.warehouse_id

        where   product_warehouse.warehouse_id not in (6,9,10)
            AND product_warehouse.is_basic_unit = 1
			and product_warehouse.available_stock > 0 

)
group by all
),
prs AS (
SELECT DISTINCT product_purchased_receipts.purchased_receipt_id,
                purchased_receipts.purchased_order_id,
                DATE_PART('Day', purchased_receipts.date::date) AS DAY,
                DATE_PART('month', purchased_receipts.date::date) AS MONTH,
                DATE_Part('year', purchased_receipts.date::date) AS YEAR,
                products.id AS product_id,
                CONCAT(products.name_ar, ' ', products.size, ' ', product_units.name_ar) AS sku,
                brands.name_ar AS Brand,
                categories.name_ar as category,
                products.description,
                purchased_receipts.warehouse_id AS warehouse_id,
                warehouses.name as warehouse,
                packing_units.name_ar AS packing_unit,
                purchased_receipts.discount AS Total_discount,
                purchased_receipts.return_orders_discount,
                purchased_receipts.discount_type_id,
                suppliers.id AS supplier_id,
                suppliers.name AS supplier_name,
                purchased_receipt_statuses.name_ar AS PR_status,
                product_purchased_receipts.basic_unit_count,
                product_purchased_receipts.purchased_item_count AS purchase_count,
                product_purchased_receipts.purchased_item_count*product_purchased_receipts.basic_unit_count AS purchase_min_count,
                product_purchased_receipts.item_price,
                product_purchased_receipts.final_price/product_purchased_receipts.purchased_item_count AS final_item_price,
                product_purchased_receipts.total_price AS purchase_price,
                CASE WHEN product_purchased_receipts.vat = 'true' THEN product_purchased_receipts.total_price * 0.14
                     ELSE CASE WHEN product_purchased_receipts.vat = 'false' THEN product_purchased_receipts.total_price * 0
                               END
                END AS vat,
                CASE WHEN purchased_receipts.discount_type_id = 2 THEN (product_purchased_receipts.discount/100) * product_purchased_receipts.total_price
                     ELSE product_purchased_receipts.discount
                END AS SKU_discount,
                purchased_receipts.total_price AS pr_value,
                CASE
                    WHEN product_purchased_receipts.t_tax_id = 1 THEN product_purchased_receipts.total_price * 0.05
                    ELSE CASE
                             WHEN product_purchased_receipts.t_tax_id = 2 THEN product_purchased_receipts.total_price * 0.08
                             ELSE CASE
                                      WHEN product_purchased_receipts.t_tax_id = 3 THEN product_purchased_receipts.total_price * 0.1
                                      ELSE 0
                                  END
                         END
                END AS table_tax,
                product_purchased_receipts.final_price AS Final_Price,
                product_purchased_receipts.product_type_id,
                purchased_receipts.debt_note_value as credit_note,
                purchased_receipts.tips,
                purchased_receipts.delivery_fees,
                case when purchased_receipts.is_actual = 'true' then 'Real' 
                     else 'Virtual' 
                     end as is_actual
                     
FROM product_purchased_receipts
LEFT JOIN products ON products.id = product_purchased_receipts.product_id
LEFT JOIN packing_unit_products ON packing_unit_products.product_id = products.id
LEFT JOIN purchased_receipts ON purchased_receipts.id = product_purchased_receipts.purchased_receipt_id
LEFT JOIN purchased_receipt_statuses ON purchased_receipt_statuses.id = purchased_receipts.purchased_receipt_status_id
LEFT JOIN packing_units ON packing_units.id = product_purchased_receipts.packing_unit_id
LEFT JOIN product_units ON products.unit_id = product_units.id
LEFT JOIN suppliers ON suppliers.id = purchased_receipts.supplier_id
LEFT JOIN brands ON brands.id = products.brand_id
left join categories on categories.id = products.category_id
left join warehouses on warehouses.id = purchased_receipts.warehouse_id
WHERE product_purchased_receipts.purchased_item_count <> 0
      AND purchased_receipts.purchased_receipt_status_id IN (4,5,7)
      AND purchased_receipts.date::date >= current_date - 4
    AND purchased_receipts.is_actual = 'true'
     
     
    ),
prs_data as (
select warehouse_id , product_id,sum(final_price) as total_prs 
from prs 
group by all
)

select warehouse_id,product_id,1 as zero_rr
from (
select s.*,
CONCAT(products.name_ar,' ',products.size,' ',product_units.name_ar) as sku,
brands.name_ar as brand, 
categories.name_ar as cat,
coalesce(md.nmv,0) as sales,wac1,
wac1*stocks as stock_value,
coalesce(total_prs,0) as prs_data
from stocks s
left join main_data md on md.product_id =s.product_id and md.warehouse_id = s.warehouse_id
JOIN finance.all_cogs f  ON f.product_id = s.product_id
                        AND f.from_date::date <= current_date 
                        AND f.to_date::date > current_date
JOIN products on products.id=s.product_id
JOIN brands on products.brand_id = brands.id 
JOIN categories ON products.category_id = categories.id
JOIN product_units ON product_units.id = products.unit_id
left join prs_data on prs_data.product_id =s.product_id and prs_data.warehouse_id = s.warehouse_id 
where stocks > 0 and sales = 0 
and prs_data < 0.7*stock_value
order by wac1* stocks desc 
)
'''
zerorr =  query_snowflake(query, columns = ['warehouse_id','product_id','zero_rr'])
zerorr.columns = zerorr.columns.str.lower()
for col in zerorr.columns:
    zerorr[col] = pd.to_numeric(zerorr[col], errors='ignore')   
zerorr    

,warehouse_id,product_id,zero_rr
0,533,24,1
1,533,26,1
2,764,6866,1
3,170,5769,1
4,533,8007,1
...,...,...,...
6938,39,12718,1
6939,1,11627,1
6940,8,12708,1
6941,170,12708,1


In [17]:
product_data = product_data.merge(product_warehouse_price,on=['product_id','warehouse_id'])
product_data = product_data.merge(uth_cntrb[['warehouse_id','uth_cntrb']],on='warehouse_id')
product_data['product_UTH_growth'] =(product_data['current_uth'] -product_data['prev_uth'])/product_data['prev_uth']
product_data['product_LH_growth'] =(product_data['current_last_hour'] -product_data['prev_last_hour'])/product_data['prev_last_hour']
product_data[['product_UTH_growth','product_LH_growth']] =product_data[['product_UTH_growth','product_LH_growth']].fillna(0) 
product_data = product_data.replace([np.inf, -np.inf], 1)
product_data['product_closing_growth'] = (product_data['product_UTH_growth']*product_data['uth_cntrb'])+(product_data['product_LH_growth']*(1-product_data['uth_cntrb']))

In [18]:
warehouse_data = product_data.groupby('warehouse_id')[['prev_all_day', 'prev_uth','prev_last_hour', 'current_all_day', 'current_uth', 'current_last_hour']].sum().reset_index()
warehouse_data['UTH_growth'] =(warehouse_data['current_uth'] -warehouse_data['prev_uth'])/warehouse_data['prev_uth']
warehouse_data['LH_growth'] =(warehouse_data['current_last_hour'] -warehouse_data['prev_last_hour'])/warehouse_data['prev_last_hour']
warehouse_data = warehouse_data.merge(uth_cntrb,on='warehouse_id')
warehouse_data['Closing_growth'] = (warehouse_data['UTH_growth']*warehouse_data['uth_cntrb'])+(warehouse_data['LH_growth']*(1-warehouse_data['uth_cntrb']))
dropping_whs = warehouse_data[warehouse_data['Closing_growth']<0]

In [19]:
growing_products  = product_data.merge(warehouse_data[['warehouse_id','UTH_growth','LH_growth','Closing_growth']],on='warehouse_id')
#needs edit
growing_products = growing_products[growing_products['product_closing_growth']>=np.maximum(growing_products['Closing_growth'],0.1)]
growing_products['max_closing'] = growing_products.groupby('product_id')['product_closing_growth'].transform(sum)
growing_products=growing_products[growing_products['max_closing']==growing_products['product_closing_growth']]
growing_products = growing_products.groupby(['product_id'])['price'].mean().reset_index()
growing_products.columns = ['product_id','maxab_good_price']

In [20]:
dropping_products = product_data.merge(dropping_whs[['warehouse_id','UTH_growth','LH_growth','Closing_growth']],on='warehouse_id')
dropping_products = dropping_products[dropping_products['product_closing_growth'] < 0]
dropping_products = dropping_products.sort_values(by='prev_all_day',ascending = False)
dropping_products = dropping_products.merge(growing_products,on='product_id',how='left')
dropping_products = dropping_products.merge(marketplace,on=['product_id','warehouse_id'],how='left')
dropping_products = dropping_products.merge(bensoliman[['product_id','ben_soliman_basic_price']],on=['product_id'],how='left')
dropping_products = dropping_products.drop(columns = 'region')
dropping_products = dropping_products.merge(scrapped_prices,on=['product_id','warehouse_id'],how='left')
dropping_products = dropping_products.drop(columns = 'region')
dropping_products = dropping_products.merge(zerorr,on=['product_id','warehouse_id'],how='left')

In [21]:
dropping_products = dropping_products.merge(warehouse_region,on=['warehouse_id'])
dropping_products = dropping_products.merge(stats,on=['product_id','region'],how='left')
dropping_products = dropping_products.merge(sku_info,on=['product_id'])
dropping_products = dropping_products.merge(brand_cat_target,on=['brand','cat'],how='left')
dropping_products = dropping_products.merge(cat_target,on=['cat'],how='left')
dropping_products['Target_margin'] = dropping_products['target_bm'].fillna(dropping_products['cat_target_margin'])
dropping_products = dropping_products[[ 'warehouse_id','product_id','sku','brand','cat', 'prev_all_day', 'prev_uth',
       'prev_last_hour', 'current_all_day', 'current_uth', 'current_last_hour','product_UTH_growth', 'product_LH_growth',
       'product_closing_growth','doh','wac_p','price','maxab_good_price', 'final_min_price', 'final_max_price',
       'final_mod_price', 'final_true_min', 'final_true_max',
       'ben_soliman_basic_price','optimal_bm', 'min_boundary',
       'max_boundary', 'median_bm','Target_margin','min_scrapped','max_scrapped','median_scrapped','zero_rr']]
dropping_products = dropping_products.merge(last_two_hours,on=['product_id','warehouse_id'],how='left')
dropping_products[['t_2_nmv','t_1_nmv','t_2_price','t_1_price']] = dropping_products[['t_2_nmv','t_1_nmv','t_2_price','t_1_price']].fillna(0)
dropping_products=dropping_products.drop_duplicates()

In [22]:
def select_price(product_UTH_growth,product_LH_growth,product_closing_growth,remaining_prices,price,wac,Target_margin,min_boundary,optimal_bm,t_1_price,zero_rr):
    target_price = 0 
    min_price = 0
    max_price = 0 
    acceptable = []
    source = ''
    current_margin = (price-wac)/price
    if not np.isnan(zero_rr):
        for i in range(0,len(remaining_prices)):
            new_price = remaining_prices[i]
            diff = (new_price-price)/price
            if new_price > wac and  diff <= -0.05:
                target_price = new_price
                source = 'Zero_rr'
                break
        if target_price == 0 and current_margin> Target_margin and current_margin - Target_margin > 0.0025:
                target_price = wac/(1-Target_margin)
                source = 'Zero_rr' 
        elif target_price == 0 and current_margin> min_boundary and current_margin - min_boundary > 0.0025:
                target_price = wac/(1-min_boundary)
                source = 'Zero_rr' 
        elif target_price == 0 and current_margin> Target_margin/2 and current_margin - Target_margin/2 > 0.0025: 
                target_price = wac/(1-(Target_margin/2))
                source = 'Zero_rr' 

    else:
        for i in range(len(remaining_prices)-1,-1,-1):
            new_price = remaining_prices[i]
            diff = (new_price-price)/price
            current_margin = (price-wac)/price
            new_margin = (new_price-wac)/new_price
            if new_margin >= min_boundary*0.9 and new_margin >= 0.3*Target_margin and new_margin >0  and diff <= -0.0025:
                target_price = new_price
                source = 'Listed'
                break
        if target_price == 0:

            for j in range(0,len(remaining_prices)):
                new_price = remaining_prices[j]
                diff = (new_price-price)/price
                current_margin = (price-wac)/price
                new_margin = (new_price-wac)/new_price
                if new_margin >0 :
                    acceptable.append(new_price)
            if(len(acceptable) > 1):
                distance_arr = []
                for k in range(0,len(acceptable)):
                    new_price = acceptable[k]
                    diff = 1/abs(price-new_price)
                    distance_arr.append(diff)

                total_array = sum(distance_arr)
                normalized = [x / total_array for x in distance_arr]
                final_value = 0 
                for v in range(0,len(normalized)):
                    w = normalized[v]
                    p = acceptable[v]
                    final_value+= (w*p)
                target_price = np.maximum(final_value,wac/(1-(0.3*Target_margin)))
                source = 'induced'

            elif (len(acceptable) > 0):
                new_price = acceptable[0]
                final_value = (0.3*new_price)+(0.7*price)
                target_price = np.maximum(final_value,wac/(1-(0.3*Target_margin)))
                source = 'induced'

              
               
    return target_price,source    
            

In [42]:
product_final_df = pd.DataFrame(columns = ['warehouse_id', 'product_id', 'sku', 'brand', 'cat', 'prev_all_day',
       'prev_uth', 'prev_last_hour', 'current_all_day', 'current_uth',
       'current_last_hour', 'product_UTH_growth', 'product_LH_growth',
       'product_closing_growth', 'doh', 'wac_p', 'price', 'maxab_good_price',
       'final_min_price', 'final_max_price', 'final_mod_price',
       'final_true_min', 'final_true_max', 'ben_soliman_basic_price',
       'optimal_bm', 'min_boundary', 'max_boundary', 'median_bm','min_scrapped','max_scrapped','median_scrapped','zero_rr',
       'Target_margin', 't_2_nmv', 't_1_nmv', 't_2_price', 't_1_price','selected_price','source'])

for _,row in tqdm(dropping_products.iterrows(), total=len(dropping_products)):
    wac = row['wac_p']
    price = row['price']
    maxab_good_price = row['maxab_good_price']
    final_min_price = row['final_min_price']
    final_max_price = row['final_max_price']
    final_mod_price = row['final_mod_price']
    final_true_min = row['final_true_min']
    final_true_max = row['final_true_max']
    ben_soliman_basic_price= row['ben_soliman_basic_price']
    scrapped_min = row['min_scrapped']
    scrapped_median = row['median_scrapped']
    scrapped_max = row['max_scrapped']
    optimal_price = wac/(1-row['optimal_bm'])
    min_b_price = wac/(1-row['min_boundary'])
    max_b_price = wac/(1-row['max_boundary'])
    median_price = wac/(1-row['median_bm'])
    target_price = wac/(1-row['Target_margin'])
    product_UTH_growth = row['product_UTH_growth']
    product_LH_growth = row['product_LH_growth']
    product_closing_growth=row['product_closing_growth']
    t_1_price = row['t_1_price']
    prices_list = [maxab_good_price,final_min_price,final_max_price,final_mod_price,final_true_min,final_true_max,ben_soliman_basic_price,optimal_price,min_b_price,max_b_price,median_price,target_price,scrapped_min,scrapped_max,scrapped_median]
    cleaned_prices = list({x for x in prices_list if x not in [0, np.nan] and not pd.isna(x)})
    if t_1_price>0 and product_UTH_growth<product_LH_growth and product_LH_growth > 0:
        row['selected_price'] = t_1_price
        row['source'] == 'Prev_disc'
    else:
        remaining_prices = [x for x in cleaned_prices if (x < price) or (t_1_price > 0 and x <= t_1_price)]
        remaining_prices.sort()
        new_price,source = select_price(product_UTH_growth,product_LH_growth,product_closing_growth,remaining_prices,price,wac,row['Target_margin'],row['min_boundary'],row['optimal_bm'],t_1_price,row['zero_rr'])
        row['selected_price'] = new_price
        row['source'] = source
        
        
    product_final_df = pd.concat([product_final_df, row.to_frame().T], ignore_index=True)



100%|██████████| 9501/9501 [00:49<00:00, 190.94it/s]


In [43]:
product_final_df['discount'] = abs((product_final_df['selected_price']-product_final_df['price'])/product_final_df['price'])
product_final_df = product_final_df[(product_final_df['discount'] > 0.0025)&(product_final_df['selected_price']>0)]
product_final_df['discount'] = product_final_df['discount']*100000
product_final_df['discount'] = ((product_final_df['discount']//10)+1)/10000
product_final_df['discount'] = np.minimum(product_final_df['discount'],0.05)
product_final_df['discount']=product_final_df['discount']*100

In [63]:
product_final_df = product_final_df[~product_final_df['zero_rr'].isna()]
product_final_df

,warehouse_id,product_id,sku,brand,cat,prev_all_day,prev_uth,prev_last_hour,current_all_day,current_uth,current_last_hour,product_UTH_growth,product_LH_growth,product_closing_growth,doh,wac_p,price,maxab_good_price,final_min_price,final_max_price,final_mod_price,final_true_min,final_true_max,ben_soliman_basic_price,optimal_bm,min_boundary,max_boundary,median_bm,min_scrapped,max_scrapped,median_scrapped,zero_rr,Target_margin,t_2_nmv,t_1_nmv,t_2_price,t_1_price,selected_price,source,discount
990,170,37,مكرونة حواء هلالية - 400 جم,حواء,مكرونة,2279.306927,1019.774605,251.781711,263.5,263.5,0.0,-0.74161,-1.0,-0.866373,11.190577,254.952357,270.5,270.25,259.0,270.0,270.0,257.99,275.0,NaN,0.030376,0.028061,0.055087,0.033164,256.0,275.0,265.0,1.0,0.040087,0.0,0.0,0.0,0.0,256.0,Zero_rr,5.0
1850,962,5648,جبنة عبور لاند شيدر - 250 جم,عبور لاند,جبن,1066.228645,790.841311,51.543045,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.695402,520.441762,539.25,NaN,535.0,548.4,535.0,535.0,550.0,NaN,0.050723,0.027048,0.0525,0.052232,539.478832,539.478832,539.478832,1.0,0.035,0.0,0.0,0.0,0.0,534.910212,Zero_rr,0.81
2048,501,11399,ارز حبوبة بسمتى زهبى هندى طويل الحبة - 1 كجم,حبوبة,أرز,935.203604,463.769064,100.214076,0.0,0.0,0.0,-1.0,-1.0,-1.0,23.0,558.282955,584.0,583.75,NaN,NaN,NaN,NaN,NaN,NaN,0.048642,0.043912,0.061244,0.048596,NaN,NaN,NaN,1.0,0.046244,0.0,0.0,0.0,0.0,571.497105,Zero_rr,2.15
2086,337,9491,هيركود جيل سبوت عرض - 10 مل,هيركود,كريم و جل للشعر,908.094889,465.76374,17.76429,0.0,0.0,0.0,-1.0,-1.0,-1.0,11.17037,12.535112,13.75,13.25,13.77,15.2,13.75,13.75,15.5,NaN,0.042102,0.036155,0.062,0.042218,NaN,NaN,NaN,1.0,0.046861,0.0,0.0,0.0,0.0,13.005316,Zero_rr,5.0
2150,170,858,هنادى زيت خليط- 2.1 لتر,هنادي,زيوت,869.047492,340.791962,18.906589,0.0,0.0,0.0,-1.0,-1.0,-1.0,46.0,562.713003,600.0,NaN,600.0,600.0,600.0,600.0,600.0,NaN,0.041615,0.041588,0.084681,0.042852,NaN,NaN,NaN,1.0,0.059411,0.0,0.0,0.0,0.0,598.256189,Zero_rr,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9496,8,20935,توداى بار كيك كراميل جامبو - 55 جم,توداي,كيك وكرواسون,0.667622,0.667622,0.0,0.0,0.0,0.0,-1.0,0.0,-0.509222,29.0,105.165,111.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046122,0.043043,0.061295,0.049345,NaN,NaN,NaN,1.0,0.046295,0.0,0.0,0.0,0.0,110.269992,Zero_rr,0.89
9497,339,12285,بن محوج غامق ( زهرة البن البرازيلى ) - 125 جم,البن البرازيلى,قهوة,0.581269,0.581269,0.0,0.0,0.0,0.0,-1.0,0.0,-0.511278,30.0,89.1,92.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.044504,0.026466,0.0525,0.041929,NaN,NaN,NaN,1.0,0.035,0.0,0.0,0.0,0.0,91.522192,Zero_rr,1.06
9498,8,807,شيفي ميكس مرقة دجاج - 18 جرام,شيفي ميكس,مرقة وخلطات,0.559916,0.559916,0.0,0.0,0.0,0.0,-1.0,0.0,-0.509222,13.0,107.51526,112.75,113.5,NaN,NaN,NaN,NaN,NaN,NaN,0.060121,0.045332,0.078696,0.054118,NaN,NaN,NaN,1.0,0.052464,0.0,0.0,0.0,0.0,110.411568,Zero_rr,2.08
9499,339,2271,ريحانة حمص شام- 500 جم,ريحانة,بقوليات,0.511499,0.511499,0.0,0.0,0.0,0.0,-1.0,0.0,-0.511278,21.0,73.029286,75.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.05137,0.034644,0.061373,0.05137,NaN,NaN,NaN,1.0,0.040915,0.0,0.0,0.0,0.0,74.554486,Zero_rr,1.58


In [64]:
product_final_df['tuple'] = product_final_df[["product_id",'warehouse_id']].apply(tuple, axis=1)
selected_skus_tuple = str(list(product_final_df['tuple']))[1:-1]
product_final_df=product_final_df.drop(columns = 'tuple')

## Retailers Selection

In [65]:
query = f'''
with selected_prods as (
select * 
from(
VALUES
{selected_skus_tuple}
)x(product_id,warehouse_id)
),
selected_districts as (
select distinct sp.warehouse_id , sp.product_id,dp.district_id  
from WAREHOUSE_DISPATCHING_RULES wdr 
join DISPATCHING_POLYGONS dp on dp.id = wdr.DISPATCHING_POLYGON_ID
join selected_prods sp on sp.product_id = wdr.product_id and wdr.warehouse_id = sp.warehouse_id
),
sales_before as (
select retailer_id,product_id,warehouse_id,district_id,avg(nmv) as avg_nmv_before
from (
SELECT  DISTINCT
so.id as order_id ,
sd.district_id,
sd.warehouse_id as warehouse_id,
pso.product_id as product_id,
so.retailer_id as retailer_id,
sum(pso.total_price) as nmv 


FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN materialized_views.retailer_polygon on materialized_views.retailer_polygon.retailer_id=so.retailer_id
JOIN districts on districts.id=materialized_views.retailer_polygon.district_id
join selected_districts sd on sd.product_id = pso.product_id and sd.district_id = districts.id
           

WHERE   True
    AND so.created_at ::date between current_date - 120 and current_date - 31
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0
	
GROUP BY ALL
)
group by all 
),
sales_after as (
select retailer_id,product_id,warehouse_id,district_id,avg(nmv) as avg_nmv_after,max(order_date) as last_order
from (
SELECT  DISTINCT
so.id as order_id ,
so.created_at::date as order_date,
sales_order_status_id, 
sd.district_id,
sd.warehouse_id as warehouse_id,
pso.product_id as product_id,
so.retailer_id as retailer_id,
sum(pso.total_price) as nmv 


FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN materialized_views.retailer_polygon on materialized_views.retailer_polygon.retailer_id=so.retailer_id
JOIN districts on districts.id=materialized_views.retailer_polygon.district_id
join selected_districts sd on sd.product_id = pso.product_id and sd.district_id = districts.id
           

WHERE   True
    AND so.created_at ::date > current_date - 31
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0
	
GROUP BY ALL
)
group by all 

),
made_order as (
select distinct so.retailer_id


FROM  sales_orders so 
JOIN materialized_views.retailer_polygon on materialized_views.retailer_polygon.retailer_id=so.retailer_id
JOIN districts on districts.id=materialized_views.retailer_polygon.district_id
join selected_districts sd on sd.district_id = districts.id
           

WHERE   True
    AND so.created_at ::date >= current_date - 60
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
	
GROUP BY ALL
)

select distinct retailer_id , product_id,warehouse_id 
from (
select sb.* , coalesce(avg_nmv_after,0) as nmv_after,(nmv_after-avg_nmv_before)/avg_nmv_before as growth
from sales_before sb 
left join sales_after sa on sb.retailer_id = sa.retailer_id and sb.product_id = sa.product_id
left join made_order mo on mo.retailer_id = sa.retailer_id 
where growth < -0.6
and (current_date - last_order >=5 or last_order is null)
and mo.retailer_id is not null 
)
'''
churned_dropped =  query_snowflake(query, columns = ['retailer_id','product_id','warehouse_id'])
churned_dropped.columns = churned_dropped.columns.str.lower()
for col in churned_dropped.columns:
    churned_dropped[col] = pd.to_numeric(churned_dropped[col], errors='ignore')      

In [66]:
query = f'''
with selected_prods as (
select * 
from(
VALUES
{selected_skus_tuple}
)x(product_id,warehouse_id)
),
selected_districts as (
select distinct sp.warehouse_id , sp.product_id,dp.district_id ,c.name_ar as cat ,b.name_ar as brand
from WAREHOUSE_DISPATCHING_RULES wdr 
join DISPATCHING_POLYGONS dp on dp.id = wdr.DISPATCHING_POLYGON_ID
join selected_prods sp on sp.product_id = wdr.product_id and wdr.warehouse_id = sp.warehouse_id
join products p on p.id = sp.product_id
join brands b on b.id = p.brand_id 
join categories c on c.id = p.category_id 
),
selected_dis_cat_brand as (
select distinct warehouse_id,district_id,cat,brand
from selected_districts
),

buy_cat as (
SELECT  DISTINCT
sd.district_id,
sd.warehouse_id as warehouse_id,
so.retailer_id as retailer_id,
c.name_ar as cat,
b.name_ar as brand,
pso.product_id


FROM product_sales_order pso
JOIN sales_orders so ON so.id = pso.sales_order_id
JOIN materialized_views.retailer_polygon on materialized_views.retailer_polygon.retailer_id=so.retailer_id
JOIN districts on districts.id=materialized_views.retailer_polygon.district_id
join products p on p.id = pso.product_id
join brands b on b.id = p.brand_id 
join categories c on c.id = p.category_id 
join selected_dis_cat_brand sd on sd.cat = c.name_ar and sd.district_id = districts.id and b.name_ar = sd.brand
           

WHERE   True
    AND so.created_at::date >= current_date - 60
    AND so.sales_order_status_id not in (7,12)
    AND so.channel IN ('telesales','retailer')
    AND pso.purchased_item_count <> 0
	

),
chosen_products as (
select sp.*,c.name_ar as cat ,b.name_ar as brand
from selected_prods sp 
join products p on p.id = sp.product_id
join brands b on b.id = p.brand_id 
join categories c on c.id = p.category_id 

)
select retailer_id,selected_product_id,warehouse_id 
from (
select warehouse_id,retailer_id,cat,brand,selected_product_id,max(flag) as flag
from (
select bc.* , cp.product_id as selected_product_id, case when cp.product_id = bc.product_id then 1 else 0 end as flag 
from buy_cat bc 
left join chosen_products cp on cp.warehouse_id = bc.warehouse_id  and cp.brand = bc.brand and cp.cat = bc.cat 
)
group by all 
)
where flag = 0 
'''
cat_not_product =  query_snowflake(query, columns = ['retailer_id','product_id','warehouse_id'])
cat_not_product.columns = cat_not_product.columns.str.lower()
for col in cat_not_product.columns:
    cat_not_product[col] = pd.to_numeric(cat_not_product[col], errors='ignore') 
cat_not_product    

,retailer_id,product_id,warehouse_id
0,470295,10428,632
1,565225,19370,501
2,204409,3720,170
3,41658,11727,962
4,162223,13271,170
...,...,...,...
308038,731173,1056,170
308039,350017,11724,797
308040,186178,7204,170
308041,264157,13855,962


In [67]:
query = '''
select retailer_id
from (
SELECT  DISTINCT
retailer_id,
sales_order_status_id,
created_at::date as o_date ,
max(o_date)over(partition by retailer_id) as last_order
from sales_orders so 
WHERE  so.created_at ::date >= current_date - 120
AND so.sales_order_status_id not in (7,12)
AND so.channel IN ('telesales','retailer')
qualify o_date = last_order
)
where sales_order_status_id <> 6 

union all 

select id as retailer_id 
from retailers 
where activation = 'false'
'''
exec_rets =  query_snowflake(query, columns = ['retailer_id'])
exec_rets.columns = exec_rets.columns.str.lower()
for col in exec_rets.columns:
    exec_rets[col] = pd.to_numeric(exec_rets[col], errors='ignore') 
exec_rets =  exec_rets.retailer_id.unique() 

In [68]:
query = '''
select distinct product_id,packing_unit_id 
from packing_unit_products
'''
pus = query_snowflake(query, columns = ['product_id','packing_unit_id'])
for col in pus.columns:
    pus[col] = pd.to_numeric(pus[col], errors='ignore')     

In [69]:
all_retailers  = pd.concat([cat_not_product, churned_dropped]).drop_duplicates().reset_index(drop=True)
all_retailers = all_retailers[~all_retailers['retailer_id'].isin(exec_rets)]

In [70]:
final_df = product_final_df.merge(all_retailers,on=['warehouse_id','product_id'])
final_df = final_df.merge(pus,on='product_id')
final_df['HH_data'] = '['+(final_df['product_id']).astype(str)+','+(final_df['packing_unit_id']).astype(str)+','+(final_df['discount']).astype(str)+']'

In [71]:
output_df  = final_df.groupby('retailer_id')['HH_data'].apply(list).reset_index()
output_df['Discounts']= output_df['HH_data'].astype(str).str.replace("'",'').str.replace(' ','')
output_df = output_df.groupby('Discounts')['retailer_id'].agg(list).reset_index()
output_df['Arabic Offer Name']= 'خصوماتك الخاصة'
output_df['Start Date/Time'] = '23/10/2025 18:00'
output_df['End Date/Time'] = '23/10/2025 23:00'
output_df = output_df[['retailer_id','Start Date/Time','End Date/Time','Discounts','Arabic Offer Name']]
output_df['French Offer Name']=np.nan
output_df['English Offer Name']=np.nan

In [72]:
data = []
for i,row in output_df.iterrows():
    
    start_date = row['Start Date/Time']
    end_date = row['End Date/Time']
    retailers = row['retailer_id']
    discount = row['Discounts']
    name = row['Arabic Offer Name'] 
    name_f = row['French Offer Name'] 
    name_e = row['English Offer Name'] 
    
    length = len(retailers)
    if(length>100):
        iters = length//100
        remaining = length%100
        for j in range(0,iters+1):
            if(j<=iters):
                start = (j*100)
                end = (j+1)*100
                rets = retailers[start:end]
                data.append({'Discounts':discount,'retailer_id':rets,'Start Date/Time':start_date,'End Date/Time':end_date
                            ,'Arabic Offer Name':name,'French Offer Name':name_f,'English Offer Name':name_e})
            else:
                print("else new")
            
    else:
        data.append({'Discounts':discount,'retailer_id':retailers,'Start Date/Time':start_date,'End Date/Time':end_date
                            ,'Arabic Offer Name':name,'French Offer Name':name_f,'English Offer Name':name_e})
        
dfx = pd.DataFrame(data)

In [73]:
dfx['English Offer Name'] = 'Special Discounts'
dfx['Swahili Offer Name'] = ''
dfx['Rwandan Offer Name'] = ''

In [74]:
dfx.rename(columns={'retailer_id': 'Retailers List'}, inplace=True)
dfx = dfx[['Retailers List','Start Date/Time','End Date/Time','Discounts','Arabic Offer Name','French Offer Name','English Offer Name','Swahili Offer Name','Rwandan Offer Name']]
# 500 row per sheet 
final=dfx.reset_index().drop(columns='index')
mino=final.index.min()
maxo=final.index.max()
ran = [i for i in range(mino,maxo,1000)]
for i in tqdm(range(len(ran))):
    if i+1 == len(ran):
        val1 = ran[i]
        val2 = maxo
    else:
        val1 = ran[i]
        val2 = ran[i+1] - 1
    x=final.loc[val1:val2,:]
    x.to_excel('HH_Sheets/o_happy_hour_{0}.xlsx'.format(i),index=False)

100%|██████████| 19/19 [00:03<00:00,  5.81it/s]


In [50]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])

In [54]:
pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
username = pricing_api_secret["egypt_username"]
password = pricing_api_secret["egypt_password"]
secret = pricing_api_secret["egypt_secret"]

In [55]:
def get_access_token(url, client_id, client_secret):
    """
    get_access_token function takes three parameters and returns a session token
    to connect to MaxAB APIs

    :param url: production MaxAB token URL
    :param client_id: client ID
    :param client_secret: client sercret
    :return: session token
    """
    response = requests.post(
        url,
        data={"grant_type": "password",
              "username": username,
              "password": password},
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]

In [60]:
def post_sku_discount(file_name):
    token = get_access_token('https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
                             'main-system-externals',
                             secret)
    url = "https://api.maxab.info/commerce/api/admins/v1/sku-discounts"
    payload={}
    files=[
      ('file',(file_name,open(file_name,'rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
    ]
    headers = {
      'Authorization': 'bearer {}'.format(token)}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    return response

In [61]:
post_sku_discount('o_happy_hour.xlsx')

<Response [404]>

In [41]:
# response = post_QD('QD_upload.xlsx')
# if response.ok:
files = [f for f in os.listdir('HH_Sheets') if os.path.isfile(os.path.join('HH_Sheets', f))]


['o_happy_hour_37.xlsx',
 'o_happy_hour_26.xlsx',
 'o_happy_hour_2.xlsx',
 'o_happy_hour_34.xlsx',
 'o_happy_hour_40.xlsx',
 'o_happy_hour_6.xlsx',
 'o_happy_hour_3.xlsx',
 'o_happy_hour_17.xlsx',
 'o_happy_hour_0.xlsx',
 'o_happy_hour_13.xlsx',
 'o_happy_hour_4.xlsx',
 'o_happy_hour_35.xlsx',
 'o_happy_hour_21.xlsx',
 'o_happy_hour_31.xlsx',
 'o_happy_hour_8.xlsx',
 'o_happy_hour_7.xlsx',
 'o_happy_hour_23.xlsx',
 'o_happy_hour_27.xlsx',
 'o_happy_hour_9.xlsx',
 'o_happy_hour_11.xlsx',
 'o_happy_hour_12.xlsx',
 'o_happy_hour_38.xlsx',
 'o_happy_hour_20.xlsx',
 'o_happy_hour_14.xlsx',
 'o_happy_hour_28.xlsx',
 'o_happy_hour_39.xlsx',
 'o_happy_hour_22.xlsx',
 'o_happy_hour_29.xlsx',
 'o_happy_hour_33.xlsx',
 'o_happy_hour_25.xlsx',
 'o_happy_hour_32.xlsx',
 'o_happy_hour_16.xlsx',
 'o_happy_hour_15.xlsx',
 'o_happy_hour_24.xlsx',
 'o_happy_hour_1.xlsx',
 'o_happy_hour_30.xlsx',
 'o_happy_hour_36.xlsx',
 'o_happy_hour_19.xlsx',
 'o_happy_hour_5.xlsx',
 'o_happy_hour_41.xlsx',
 'o_happy_